In [ ]:
%load_ext kamu

In [ ]:
import kamu

import os
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import hvplot.pandas
import hvplot.xarray
from datetime import datetime
from mapboxgl.utils import create_color_stops, create_numeric_stops, df_to_geojson
from mapboxgl.viz import CircleViz

con = kamu.connect()

In [ ]:
%%sql
select * from 'rijkswaterstaat.nl/stations' limit 3

In [ ]:
%%sql
select * from 'rijkswaterstaat.nl/measurements.boven-rijn' limit 3

# Visualizing Water Stations Data
Here we show the dataset using a scatterplot. We select the 25th timestep, which corresponds to 2020-01-01 00:00 . We visualize the waterlevels for that timestep, using a scatterplot with 1 value per station.

In [ ]:
%%sql -o df -q
select
    m.event_time,
    m.station_id,
    s.station_name, 
    s.lat,
    s.lon,
    waterlevel,
    velocity,
    discharge
from 'deltares.nl/rhine-basin.netherlands' as m
left join 'rijkswaterstaat.nl/stations' as s 
    on m.station_id = s.station_id

In [ ]:
ds = df.set_index(['event_time', 'station_id']).to_xarray()
ds.plot.scatter(x='lon', y='lat', hue='waterlevel', edgecolors='none')

In [ ]:
viz = CircleViz(
    df_to_geojson(
        df, properties=['station_id', 'station_name', 'waterlevel'], lat='lat', lon='lon', precision=3
    ),
    access_token=os.getenv('MAPBOX_ACCESS_TOKEN'),
    height='500px',
    radius=3,
    stroke_width = 0,
    color_property = "waterlevel",
    color_stops = create_color_stops([0, 2.5, 5, 7.5, 10], colors='YlOrRd'),
    center = (5.7, 52.2),
    zoom = 7.8,
    #below_layer = 'waterway-label',
    style='mapbox://styles/mapbox/streets-v11',
)
viz.show()

# Simulating Predicted Water Levels

In [ ]:
%%sql -o df2 -q
select 
    analysis_time, 
    sim_time,
    waterlevel,
    velocity,
    discharge,
    round((cast(sim_time as bigint) - cast(analysis_time as bigint))/3600) as lookahead
from 'deltares.nl/rhine-basin.netherlands.sim' as m
where 
    station_id = 'BR_0863.00'
    and waterlevel is not null
order by analysis_time, sim_time

In [ ]:
df2.hvplot.scatter(
    x='sim_time', y=['waterlevel', 'velocity', 'discharge'], shared_axes=False, c='lookahead', 
    cmap='magma', s=2, height=300, width=800, subplots=True
).cols(1)

# Compute statistics
Here we take two years of forecasts to compute  the discharge - waterlevel/velocity relation. We take the discharge at Lobith and the waterlevel and velocity at Thiel. We will start by looking at one forecast series. We can select the discharge and take the waterlevel 12 hours later. 

In [ ]:
%%sql -o lobith -q
select
    analysis_time,
    sim_time,
    discharge
from 'deltares.nl/rhine-basin.netherlands.sim'
where station_id = 'BR_0863.00'
order by analysis_time, sim_time

In [ ]:
%%sql -o thiel -q
select
    analysis_time,
    sim_time,
    waterlevel,
    velocity
from 'deltares.nl/rhine-basin.netherlands.sim'
where station_id = 'WA_0913.00'
order by analysis_time, sim_time

Now we can select waterlevels, discharge and velocities for all timesteps. We could decide to filter by time difference with analysis time. We could compare 12 hours ahead with the analysis time. For now let's use all the data (all time combinations). 

Let's visualize the data by showing the histograms.

In [ ]:
lds = lobith.to_xarray()
tds = thiel.to_xarray()

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(13, 6))
axes[0].hist(lds.discharge.values.ravel())
axes[1].hist(tds.waterlevel.values.ravel())
axes[2].hist(tds.velocity.values.ravel())
fig

Show the Q-h and Q-v relationship. Now we just do it for 1 location, but we need it for all locations. We also need to incorporate the time lag and to fit a non-parametric monotonic increasing function. But that's for a later moment. For now let's just inspect the functions.

In [ ]:
fig, ax = plt.subplots(figsize=(13, 8))
ax.plot(lds.discharge.values.ravel(), tds.waterlevel.values.ravel(), 'k.', alpha=0.1)
ax.set_xlabel('Discharge @ Lobith [m3/s]')
ax.set_ylabel('Waterlevel @ Thiel [m]')
fig

In [ ]:
%%local
fig, ax = plt.subplots(figsize=(13, 8))
ax.plot(lds.discharge.values.ravel(), tds.velocity.values.ravel(), 'k.', alpha=0.1)
ax.set_xlabel('Discharge @ Lobith [m3/s]')
ax.set_ylabel('Velocity @ Thiel [m]')
fig